# Clustering
---

This workflow demonstrates how to use the API to fetch trajectories and perform RMSD-based clustering for a specified number of clusters.

### Workflow Steps
1. **Download structure and trajectory files** using the API.  
2. **Compute the RMSD matrix** for the selected trajectories.  
3. **Perform iterative RMSD-based clustering** with a dynamic cutoff, ensuring the final number of representative structures falls within the desired range.

This pipeline is employed in the paper [Mokhtari et al., 2025](https://doi.org/10.1101/2025.03.04.641377) as a pre-processing step.

---

For more information about this database, see our related work [*DynaRepo: The Repository of Macromolecular Conformational Dynamics*](https://doi.org/10.1101/2025.08.14.670260).

**Authors:**  
Omid Mokhtari, Emmanuelle Bignon, Hamed Khakzad, Yasaman Karami (Nancy, France)

**For correspondence:** yasaman.karami@inria.fr

---

#### Import Required Libraries

In [1]:
import json, urllib
from os.path import exists
import mdtraj as md
import numpy as np
import glob
import os

In [2]:
API_BASE_URL = "http://inria.mddbr.eu/api/rest/current"

#### Functions

In [3]:
def query_api (url : str) -> dict:
    parsed_url = url.replace(" ", "%20")
    with urllib.request.urlopen(parsed_url) as response:
        return json.loads(response.read().decode("utf-8"))
def download_file_api (url : str, filename : str):
    parsed_url = url.replace(" ", "%20")
    urllib.request.urlretrieve(url, filename)
    
def gromos(traj, rmsd_matrix, cutoff, atom_indices=None):
    n_frames = traj.n_frames
    
    # GROMOS clustering algorithm
    clusters = np.full(n_frames, -1, dtype=int)  # -1 means unassigned
    cluster_centers = []
    cluster_id = 0
    
    unassigned = np.arange(n_frames)
    
    while len(unassigned) > 0:
        neighbor_counts = np.zeros(len(unassigned))
        
        for i, frame_idx in enumerate(unassigned):
            distances = rmsd_matrix[frame_idx, unassigned]
            neighbor_counts[i] = np.sum(distances <= cutoff)
        
        center_idx = unassigned[np.argmax(neighbor_counts)]
        cluster_centers.append(center_idx)
        
        distances_to_center = rmsd_matrix[center_idx, unassigned]
        within_cutoff = unassigned[distances_to_center <= cutoff]
        
        clusters[within_cutoff] = cluster_id
        
        unassigned = np.setdiff1d(unassigned, within_cutoff)
        cluster_id += 1
    
    return clusters, np.array(cluster_centers)

#### Download both structure and trajectory data

Change the project name

In [4]:
project_name = 'A00K6'

# Set the structure query URL for the API
specific_project_url = API_BASE_URL + f'/projects/{project_name}'
structure_query = specific_project_url + '/files/structure'
print('We query the API at ' + structure_query)

# Download the file with an arbitrary name
structure_filename = 'structure.pdb'
download_file_api(structure_query, structure_filename)
if exists(structure_filename):
    print('Structure file has been downloaded successfully')

# Set the structure query URL for the API
trajectory_query = specific_project_url + '/files/trajectory?format=xtc'
print('We query the API at ' + trajectory_query)

# Download the file with an arbitrary name
trajectory_filename = 'trajectory.xtc'
print('This may take a few seconds...', end='\r')
download_file_api(trajectory_query, trajectory_filename)
if exists(trajectory_filename):
    print('Trajectory file has been downloaded successfully')

We query the API at http://inria.mddbr.eu/api/rest/current/projects/A00K6/files/structure
Structure file has been downloaded successfully
We query the API at http://inria.mddbr.eu/api/rest/current/projects/A00K6/files/trajectory?format=xtc
Trajectory file has been downloaded successfully


#### RMSD calculation

In [5]:
traj = md.load(trajectory_filename, top=structure_filename)
# Select C-alpha atoms (like GROMACS group 3)
atom_indices = traj.topology.select('name CA')


# Compute RMSD matrix
n_frames = traj.n_frames
rmsd_matrix = np.zeros((n_frames, n_frames))
for i in range(n_frames):
    rmsd_matrix[i] = md.rmsd(traj, traj, frame=i, atom_indices=atom_indices)

#### Clustering

In [6]:
# Adaptive GROMOS clustering

cutoff = 0.15
max_clusters = 100
min_clusters = 10

while True:
    clusters, cluster_centers = gromos(traj, rmsd_matrix, cutoff=cutoff, atom_indices=atom_indices)
    num_clusters = len(np.unique(clusters))

    if min_clusters <= num_clusters <= max_clusters:
        break
    elif num_clusters < min_clusters:
        cutoff -= 0.01
    elif num_clusters > max_clusters:
        cutoff += 0.01

    if cutoff < 0.05 or cutoff > 1:
        print("Cutoff out of range, clustering failed.")
        break

print(f"Clustering completed with cutoff {cutoff:.2f} and {num_clusters} clusters.")

# Save cluster representative structures
for i, center in enumerate(cluster_centers):
    traj[center].save_pdb(f"cluster_{i}.pdb")

print("Cluster representative structures have been saved.")

Clustering completed with cutoff 0.32 and 95 clusters.
Cluster representative structures have been saved.


#### Merging representative conformations

In [7]:
print("Merging cluster representatives and cleaning up the directory...")
cluster_files = sorted(glob.glob("cluster_*.pdb"), key=lambda x: int(x.split('_')[1].split('.')[0]))
trajectories = [md.load(f) for f in cluster_files]
combined = md.join(trajectories)
combined.save_pdb("representatives.pdb")
for f in cluster_files:
    os.remove(f)
print(f"Merged {len(cluster_files)} clusters into cluster_representatives.pdb and cleaned up individual files.")

Merging cluster representatives and cleaning up the directory...
Merged 95 clusters into cluster_representatives.pdb and cleaned up individual files.


# Curious or stuck? Drop me a line:

Omid.mokhtari@inria.fr